In [42]:
import pandas as pd
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [2]:
A = pd.read_csv("../assets/data/cleaned/A.csv", index_col=0)
B = pd.read_csv("../assets/data/cleaned/B.csv", index_col=0)
C = pd.read_csv("../assets/data/cleaned/C.csv", index_col=0)

In [3]:
B.shape, C.shape, A.shape

((1111, 24), (1334, 24), (1220, 24))

In [4]:
def stat_summary(X, iqrl=0.25, iqrr=0.75):
    '''
    Return for each timestamp (hr) return min, max, 0.25 quantile, 0.75 quantile, mean, var
    '''
    return pd.DataFrame(np.array([np.min(X, axis=0), np.quantile(X, axis=0, q=iqrl), np.median(X, axis=0), 
          np.quantile(X, axis=0, q=iqrr), np.max(X, axis=0), np.mean(X, axis=0), np.var(X, axis=0), np.std(X, axis=0)]).transpose(),
                        columns=["Min", "0.25Q", "Med", "0.75Q", "Max", "Mean", "Var", "StD"])

def trend(X, span = None):
    '''
    Return EWMA trend for signal samples
    '''
    if not span:
        span = X.shape[1]
    return pd.DataFrame(X).apply(lambda x: x.ewm(span=span).mean(), axis=1)

def train_test_val_split(X, y=None, test_size=0.2,  val_size=0.4):
    from sklearn.model_selection import train_test_split
    if y.__len__() > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
        X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_size)
        return X_train, X_test, X_val, y_train, y_test, y_val
    else:
        X_train, X_test = train_test_split(X, test_size=test_size)
        X_test, X_val = train_test_split(X_test, test_size=val_size)
        return X_train, X_test, X_val

def reshapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(-1,1))
    return np.array(X_)

def shapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(1,-1)[0])
    return np.array(X_)


def meanShift(X, window=None):
    '''
    Apply MA on convolved signal and compute rs means
    '''
    # At this point X must be EWMA convolved already
    if not window:
        window = X.shape[1] // 2
    return pd.DataFrame(X).apply(lambda x: x.rolling(window=window).mean(), axis=1).iloc[:,window-1:]

def gradient(X, shift=1, threshold=0):
    '''
    First order gradient of signal. Eq to diff for discrete signal
    '''
    return pd.DataFrame(X).apply(lambda x: x.diff(periods=shift), axis=1).iloc[:,threshold:]

def bucketPeaks(peakIndex):
    '''
    Bucket into first half and second half
    '''
    return(sum(peakIndex <= 12), sum(peakIndex > 12)) 

In [80]:
X = np.row_stack((A,B,C))
Y = [0]*(A.__len__()) + [1]*(B.__len__()) + [2]*C.__len__()

In [93]:
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, to_categorical(Y), test_size=0.2)

In [94]:
X_train, y_train = RandomUnderSampler().fit_resample(X_train, y_train)

C:\Users\sabri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [95]:
np.sum(y_train, axis=0)

array([886, 886, 886])

In [97]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, np.argmax(y_train, axis=1))

RandomForestClassifier(random_state=42)

In [98]:
confusion_matrix(np.argmax(y_test, axis=1),rf.predict(X_test))

array([[ 89,  38,  11],
       [ 51,  65,  22],
       [ 19,  25, 119]], dtype=int64)

In [99]:
print(classification_report(np.argmax(y_test, axis=1),rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.56      0.64      0.60       138
           1       0.51      0.47      0.49       138
           2       0.78      0.73      0.76       163

    accuracy                           0.62       439
   macro avg       0.62      0.62      0.61       439
weighted avg       0.63      0.62      0.62       439



LSTM

In [102]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=(24, 1)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [103]:
X_train.shape

(2658, 24)

In [104]:
model.fit(reshapeInput(X_train), y_train, 
          epochs=100, batch_size=50,
          validation_data=(reshapeInput(X_val), y_val),
         callbacks=tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True))

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
51/54 [===========================>..] - ETA: 0s - loss: 1.1012 - accuracy: 0.3309WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000212D8471048> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
54/54 [==============================] - 2s 13ms/step - loss: 1.1009 - accuracy: 0.3317 - val_loss: 1.0941 - val_accuracy: 0.3435
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 1.0925 - accuracy: 0.3788 - val_loss: 1.0805 - val_accuracy: 0.4966
Epoch 3/100
54/54 [==============================] - 0s 6ms/step - loss: 1.0787 - accuracy: 0.4363 - val_loss: 1.0493 - val_accuracy: 0.5136
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: 1.0571 - accuracy: 0.4515 - val_loss: 1.0259 - val_accuracy: 0.5136
Epoch 5/100
54/54 [==============================] - 0s 6ms/step - loss: 1.0266 - accuracy: 0.4817 - val_loss: 0.9757 - val_accuracy: 0.

54/54 [==============================] - 0s 6ms/step - loss: 0.7310 - accuracy: 0.6797 - val_loss: 0.6870 - val_accuracy: 0.7041
Epoch 56/100
54/54 [==============================] - 0s 6ms/step - loss: 0.7501 - accuracy: 0.6661 - val_loss: 0.6622 - val_accuracy: 0.7007
Epoch 57/100
54/54 [==============================] - 0s 6ms/step - loss: 0.6998 - accuracy: 0.6979 - val_loss: 0.6532 - val_accuracy: 0.7211
Epoch 58/100
54/54 [==============================] - 0s 6ms/step - loss: 0.6893 - accuracy: 0.6955 - val_loss: 0.6165 - val_accuracy: 0.7449
Epoch 59/100
54/54 [==============================] - 0s 6ms/step - loss: 0.6530 - accuracy: 0.7199 - val_loss: 0.6493 - val_accuracy: 0.7517
Epoch 60/100
54/54 [==============================] - 0s 6ms/step - loss: 0.7091 - accuracy: 0.6842 - val_loss: 0.6890 - val_accuracy: 0.7177
Epoch 61/100
54/54 [==============================] - 0s 6ms/step - loss: 0.6968 - accuracy: 0.6951 - val_loss: 0.6817 - val_accuracy: 0.6939
Epoch 62/100
54/54 

In [110]:
confusion_matrix(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test)))

C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[125,   8,   5],
       [ 49,  71,  18],
       [ 15,   5, 143]], dtype=int64)

In [111]:
print(f"Overall accuracy of ABC Model: {accuracy_score(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test)))}")

Overall accuracy of ABC Model: 0.7722095671981777


C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


model.save("../assets/models/ABC_discriminant")

<b> Final thoughts before wrapping up: <br/>
    1. Given more time, I would have liked to investigate signal feature extraction methods insead of temporal sequence based predictions
    2. Our effort to clean the polluted data has raised the overall accuracy of discriminating ABC from 42 % for unclean data to 77 % for clean data, though as can be clearly seen, we could increase the discrimination capability of model for A vs B further, as given B datapoints, we confound 49 of them with A meaning data A has some identical temporal patterns